### Visualize 2D obstacle in joint space
 for middle finger, add two points as obstacles

In [ ]:
import numpy as np
import time
import torch

import sys
sys.path.append('../')
from NN_model.nn_model_eval import NN_hand_obj

%load_ext autoreload
%autoreload 2


In [ ]:
g = [2] # middle finger

use_cuda = True
nn = NN_hand_obj(g=g, path_prefix_suffix=['../NN_model/models/single_', '01.pt'], use_cuda=use_cuda)

In [4]:
# generate data for the movement of the obstacle and grids for two joints

lb = nn.hand_bound[0, (g[0] - 1) * 4 + 1:g[0] * 4]
ub = nn.hand_bound[1, (g[0] - 1) * 4 + 1:g[0] * 4]

nums = 300

x_ = np.linspace(lb[0], ub[0], nums)
y_ = np.linspace(lb[1], ub[1], nums)

x, y= np.meshgrid(x_, y_, )

x1 = x.flatten().reshape(-1, 1)
y1 = y.flatten().reshape(-1, 1)

q = np.concatenate([np.zeros([nums ** 2, 1]), x1, y1, np.zeros([nums ** 2, 1])], axis=1)
q_gpu = torch.Tensor(q).to('cuda:0') if use_cuda else torch.Tensor(q)
print('Generate joint grid samples done!')


# x0 = np.array([[0.07, 0.044, 0.2]])
# up [7.90000000e-02  0  1.97000000e-01]
# down [9.70000000e-02  0  1.29000000e-01]


for step in range(400):
    num_obj = 1
    # x_obj = np.array([[7.50000000e-02,  0 , 1.87000000e-01], 
    #                 [9.70000000e-02,  0 , 1.29000000e-01]])
    # x_obj = np.array([[7.90000000e-02,  0 , 1.97000000e-01], 
    #                 [2.90000000e-02,  0 , 1.99000000e-01]])
    x_obj = np.array([[7.90000000e-02,  0 , 1.97000000e-01], 
                    ])
    # x_obj = np.random.uniform(np.array([0,0,0]), np.array([0.1,0,0.3]), size=(1, 3))
    # x_obj = np.array([[7.90000000e-02,  0 , 1.97000000e-01], 
    #                 [6.40000000e-02,  0 , 1.49000000e-01]])
    # x_obj = np.array([[8.7000000e-02,  0 , 1.97000000e-01], 
    #                 [6.0000000e-02,  0 , 1.5000000e-01]])  # connected two obstacles
    # x_obj = np.array([[7.90000000e-02,  0 , 1.97000000e-01], 
    #                 [6.40000000e-02,  0 , 1.19000000e-01]])
    # x_obj = np.array([ 
    #                 [2.90000000e-02,  0 , 1.99000000e-01]])

    x_obj += np.array([1.8e-2 * np.sin(step/20), 0, 0])

    x_obj_tensor = torch.Tensor(x_obj).to('cuda:0') if use_cuda else torch.Tensor(x_obj)
    # use the sum of distance to calculate gradient
    n = q.shape[0]

    t0 = time.time()
    # This is for getting the unnormalized distance, which is used for isoline visualization

    dis2 = np.zeros([n, num_obj])
    grad2_ = []
    for i in range(num_obj):  # for each position of the obstacle
        dis_, grad_ = nn.eval_multiple(q_gpu[0:n], x_obj_tensor[i:i+1,:], real_dis=True, only_min_dis=True, gradient=True)
        dis2[:,i] = dis_.flatten()
        grad2_.append(grad_)

    index_min = np.argmin(dis2, 1)
    dis2 = dis2[range(0,dis2.shape[0]),index_min] # only use the min distance
    # grad2 = np.asarray(grad2)
    grad2 = np.zeros([n,4])
    for i in range(n):
        grad2[i,:] = grad2_[index_min[i]][i,:]



    t1 = time.time() - t0

    print('time cost:', t1)
    n = q.shape[0]
    dis_unnor_3D = np.zeros([nums**2])


    dis1 = np.zeros([n, num_obj])
    # dis_unnor_3D = np.zeros([nums**2, num_obj_move])
    for i in range(num_obj):
        dis_, _ = nn.eval_multiple(q_gpu[0:n], x_obj_tensor[i:i+1,:], real_dis=False, only_min_dis=True, gradient=True)
        dis1[:,i] = dis_.flatten()

    index_min = np.argmin(dis1, 1)
    dis1 = dis1[range(0,dis2.shape[0]),index_min]
    dis_unnor_3D = dis1


    dis = nn.eval_multiple(q_gpu, x_obj_tensor, real_dis=False, only_min_dis=True, gradient=False)

    def vel_once(q_, dis, grad_):
        dq = ds_0.eval(q_)
        gamma = (dis - safety_margin) * dis_scale + 1
        M = modify_DS.get_M(grad_, gamma, rho=1)
        dq = M @ dq
        return dq[1:3]
    
    q_now = np.array([0, 0.25,0.25,0])
    q_goal = np.array([0, 1, 0.25, 0])
    from DS import linear_system, Modulation
    import tqdm

    ds_0 = linear_system(q_goal)
    modify_DS = Modulation(4)

    safety_margin = 0.001  # meter
    dis_scale = 100
    rho = 1

    def vel(j):  # single loop of multiple processing
        # dq_all = np.zeros([nums**2, 2])

        dq = ds_0.eval(q[j,:])
        gamma = (dis2[j] - safety_margin) * dis_scale + 1
        M = modify_DS.get_M(grad2[j,:], gamma, dq=dq, rho=rho)
        dq = M @ dq
            # dq_all[i,:] = dq[1:3]    
        return dq[1:3]

    def cal_vel(num): # use multiple processing to speed it up
        from multiprocessing import Pool
        from contextlib import closing
        # Memory usage keep growing with Python's multiprocessing.pool
        # use this to close them
        # with Pool(40) as a_pool:
        with closing(Pool(10)) as a_pool: # use 10, in case out of memory
            result = list(tqdm.tqdm(a_pool.imap(vel, range(num)), total= num))

        # result = np.vstack(result)  # a list of array to array
        return result

    # vel_3D = cal_vel(nums**2 * num_obj_move)
    vel_3D = cal_vel(n)
    vel_3D = np.vstack(vel_3D)

    import matplotlib.pyplot as plt
    plt.rcParams["font.family"] = "Times New Roman"
    plt.rcParams["font.size"] = "8"
    from matplotlib import rc

    rc('text', usetex=True)
    level = np.linspace(-1,1,11)
    import os
    suffix = str(safety_margin)
    path = 'figures/isoline_2_obj_middle_' + suffix + '_rho_' + str(rho) + '_tail' + '_goal2'
    os.makedirs(path+ '/', exist_ok=True)

    # for j in range(n): # will be out of memory
    # for j in range(1):
    fig_width = 90/25.4  * 1
    fig, ax = plt.subplots(figsize=(fig_width, fig_width ))

    output = dis1.reshape(nums, nums) # use the distances without normalization
    vel_z = vel_3D.reshape(nums, nums, 2)

    con = plt.contourf(x, y, output, level, cmap='PiYG')

    plt.contour(con, levels=[0], colors=('k',),linestyles=('-',),linewidths=(2,))  # boundary of the obs in C space
    plt.title(label='Isolines and DS streamlines')
    plt.title(label='Isolines')
    # plt.title(label=str(x_obj[0,:]))

    cax = plt.axes([0.95, 0.1, 0.05, 0.8])
    plt.colorbar(con, cax=cax)
    plt.title(label='Dis' )
    ax.scatter(q_now[1], q_now[2], c = 'b', s=10)
    ax.scatter(q_goal[1], q_goal[2], c = 'r', s = 10)

    # ax.streamplot(x[:,:], y[:,:], vel_z[:,:,0], vel_z[:,:,1], density=[0.8, 0.8], linewidth=1, minlength=0.15) # screamlines for DS in 2D
    ax.set_xlabel('Middle $q_2$'+' (rad)')  
    ax.set_ylabel('Middle $q_3$'+' (rad)')
    ax.set_xlim(nn.hand_bound[:,1])
    ax.set_ylim(nn.hand_bound[:,2])
    ax.set_aspect('equal', adjustable='box')


    path = 'figures'
    

    suffix =  '.jpg'
    save_name = path +'/' + 'DS_' + str(step) + suffix
    # while os.path.isfile(save_name):
        # save_name = save_name[:-7] + str(int(save_name[-7:-4]) + 1) + suffix

    if 'pdf' in suffix:
        fig.savefig(save_name,format=suffix[1:],bbox_inches='tight',  pad_inches=0.0)
    else:
        fig.savefig(save_name,format=suffix[1:],bbox_inches='tight',  pad_inches=0.0, dpi=600)
    plt.close()

Generate joint grid samples done!
time cost: 0.05947613716125488


100%|██████████| 90000/90000 [00:03<00:00, 26336.16it/s]


time cost: 0.09042143821716309


100%|██████████| 90000/90000 [00:03<00:00, 25939.72it/s]


time cost: 0.0633237361907959


...

100%|██████████| 90000/90000 [00:03<00:00, 24581.79it/s]


time cost: 0.0888681411743164


 58%|█████▊    | 51889/90000 [00:02<00:01, 25695.01it/s]


KeyboardInterrupt: 

Process ForkPoolWorker-972:
Process ForkPoolWorker-970:
Process ForkPoolWorker-969:


In [ ]:
dis2.shape